In [7]:
import numpy as np
from numpy.linalg import norm, eig
from functools import partial
from algorithms import conjugate_gradient, secant, Finite_Difference


In [ ]:
# def armijo(x,
#            cost_function,
#            gradient,
#            search_dir,
#            gamma=1.5,
#            r = 0.8,
#            log=True):
    
#     def v_bar(cost_x,grad_x_s,w):
#         return cost_x + 0.5*w*grad_x_s

#     w = 1
#     cost_x = cost_function(x)
#     grad_x_s = np.dot(gradient,search_dir)
#     # initialize p
#     p = 0
#     # propogate forward
#     while cost_function(x + (gamma**p)*search_dir) < v_bar(cost_x, grad_x_s, gamma**p): 
#         w = gamma**p
#         # increment p
#         p += 1
#     # initialize q
#     q = 0
#     # propogate backwards
#     while cost_function(x + (r**q * gamma**p)*search_dir) > v_bar(cost_x, grad_x_s, r**q * gamma**p): 
#         # increment q
#         q += 1
#         # consider step size w
#         w = r**q * gamma**p

#     # return step size
#     if log:
#         print(f'p={p}, q={q}, w={w}')
#     return w


In [1]:




def steepest_descent1(x0,
                     cost_function,
                     gradient_function = None,
                     step_size = 'armijo',
                     threshold = 1e-4, 
                     log = False, 
                     h = 1e-8, 
                     max_iter = 1e6, 
                     gamma = 1.5, 
                     r = 0.8, 
                     fd_method = 'central', 
                     track_history = False):

    #if no gradient function available, use finite difference appx
    if gradient_function == None: 
        fd = Finite_Difference(cost_function, fd_method, h)
        gradient_function = fd.estimate_gradient

    x_history, V_history = [],[]
    #initialize iterator, x, and gradient
    i = 0
    x = x0
    gradient = gradient_function(x)
    minimum = cost_function(x)

    #iterate until near zero gradient or max iterations reached
    while norm(gradient) >= threshold and i <= max_iter: 

        #update gradient
        gradient = gradient_function(x)

        #determine step size
        if step_size == 'armijo':
            step_size = armijo(x, cost_function, gradient, gamma, r, log = False)
        elif isinstance(step_size, (int, float)):
            pass
        else: 
            raise ValueError('step size should be float, int or "armijo"')
        
        # move to a new x by moving from the original x in the negative
        # direction of the gradient according to a given step size
        x = x - step_size*gradient
        minimum = cost_function(x)

        #result tracking
        i += 1
        if log and i % 1e4 == 0: 
            print(f'x = {x}, V(x) = {minimum:.5f}')
        if track_history: 
            x_history.append(x), V_history.append(minimum)

    if track_history:
        return x, minimum, x_history, V_history
    else: 
        return x, minimum



In [8]:
def armijo(x, cost, grad, s, gamma=1.5, mu=0.8):
    w = 1  # Default step size

    k_g = 0  # Power of gamma
    k_m = 0  # Power of mu

    # Precompute cost and gradient to save time
    vx = cost(x)
    gx_s = np.dot(grad(x), s)

    def v_bar(w):
        return vx + 0.5 * w * gx_s

    while p.cost(x + gamma**k_g * s) < v_bar(gamma**k_g):
        k_g += 1
        w = gamma**k_g

    while p.cost(x + mu**k_m * gamma**k_g * s) > v_bar(mu**k_m * gamma**k_g):
        k_m += 1
        w = mu**k_m * gamma**k_g

    return w

def steepest_descent(p, x, tol=1e-6, max_iter=999, hist=False):
    i = 0
    x_hist = []
    while np.linalg.norm(p.grad(x)) > tol:
        if i > max_iter:
            break
        s = -p.grad(x).T
        w = _step_size(p, x, s)
        x_hist.append(x)
        x = x + w * s
        i += 1

    return x if not hist else np.array(x_hist)

def _fd_grad(f, x, h=1e-8):
    dim = np.max(np.shape(x))
    grad_gen = ((f(x + h * np.eye(dim)[:, [i]]) - f(x)) / h
               for i in range(0, dim))
    grad = np.expand_dims(np.fromiter(grad_gen, np.float64), axis=0)
    return grad


class Problem:
    """Optimization problem"""
    def __init__(self, cost, grad=None, grad_step=None,
                 eq_const=None, ineq_const=None):
        self._cost = cost
        # Check presence of gradient function
        if grad is None and grad_step is None:
            # No gradient function, default step size
            self._grad_step = 1e-8
            self._grad = partial(_fd_grad, self.cost, h=self._grad_step)
        elif grad is None and grad_step is not None:
            # No gradient function, specified step size
            self._grad_step = grad_step
            self._grad = partial(_fd_grad, self.cost, h=self._grad_step)
        elif grad is not None:
            # Gradient given, no need for step size
            self._grad_step = None
            self._grad = grad
        # Check presence of constraints
        if eq_const is not None:
            self._eq_const = eq_const
        else:
            self._eq_const = None
        if ineq_const is not None:
            self._ineq_const = ineq_const
        else:
            self._ineq_const = None

    def cost(self, x=None):
        if x is not None:
            return self._cost(x)
        else:
            return self._cost

    def grad(self, x=None):
        if x is not None:
            return self._grad(x)
        else:
            return self._grad

    def eq_const(self, x=None):
        if self._eq_const is not None:
            if x is not None:
                return np.array([[eq(x)] for eq in self._eq_const])
            else:
                return np.array([eq for eq in self._eq_const])
        else:
            return None

    def ineq_const(self, x=None):
        if self._ineq_const is not None:
            if x is not None:
                return np.array([[ineq(x)] for ineq in self._ineq_const])
            else:
                return np.array([ineq for ineq in self._ineq_const])
        else:
            return None

    def num_eq_const(self):
        """Returns number of equality constraints"""
        if self._eq_const is not None:
            return np.max(np.shape(self._eq_const))
        else:
            return 0

    def num_ineq_const(self):
        """Returns number of inequality constraints"""
        if self._ineq_const is not None:
            return np.max(np.shape(self._ineq_const))
        else:
            return 0


In [4]:


def augmented_lagrange(x0,
                        cost_function, 
                        gradient_function,
                        equality_constraints, 
                        inequality_constraints,
                        tol=1e-6,
                        tol_const=1e-6, 
                        sigma_max=1e12,
                        hist=False):

    def phi(x):
        """Unconstrained problem"""
        cost = p.cost(x)

        n_e = p.num_eq_const()
        n_i = p.num_ineq_const()
        n_c = n_e + n_i

        lmb_e = lmb[0:n_e, :]
        lmb_i = lmb[n_e:n_c, :]
        sgm_e = sgm[0:n_e, :]
        sgm_i = sgm[n_e:n_c, :]


        if p.eq_const() is not None:
            c_e = p.eq_const(x)
            cost = cost - sum(lmb_e * c_e) + 0.5 * sum(sgm_e * c_e**2)
        if p.ineq_const() is not None:
            c_i = p.ineq_const(x)
            p_i = np.array([-lmb_i[i] * c_i[i] + 0.5 * sgm_i[i] * c_i[i]**2 
                            if c_i[i] <= lmb_i[i] / sgm_i[i] 
                            else -0.5 * lmb_i[i]**2 / sgm_i[i] 
                            for i in range(0, n_i)])
            cost = cost + sum(p_i)
        return cost

    p = Problem(cost_function,
                gradient_function,
                eq_const=equality_constraints,
                ineq_const=inequality_constraints)

    x_hist = []

    n_e = p.num_eq_const()
    n_i = p.num_ineq_const()
    n_c = n_e + n_i

    lmb = np.zeros((n_c, 1))
    sgm = np.ones((n_c, 1))

    x = x0
    c = 1e12 * np.ones((n_c, 1))

    while np.linalg.norm(c) > tol_const:
        # Create new problem to solve, but unconstrained
        
        up = Problem(partial(phi))

        print(phi(x))


        x_hist.append(x)
        x = steepest_descent(up, x0, tol=tol)
        x,_ = steepest_descent1(x0,
                                phi,
                                gradient_function = None,
                                max_iter = 999,
                                step_size = 'armijo')


        # Concatenate costs
        c_prv = c
        c_e = p.eq_const(x)
        c_i = p.ineq_const(x)
        if c_e is not None and c_i is not None:
            c = np.concatenate((c_e, c_i), axis=0)
        elif c_e is not None:
            c = c_e
        elif c_i is not None:
            c = c_i

        # Make sure sigma is not too big
        if any(sgm >= sigma_max):
            break

        # Update sigma
        if np.linalg.norm(c, np.inf) > 0.25 * np.linalg.norm(c_prv, np.inf):
            for i in range(0, n_c):
                if np.abs(c[i]) > 0.25 * np.linalg.norm(c_prv, np.inf):
                    sgm[i] *= 10
            continue

        lmb = lmb - (sgm * c)
    print(x)
    return x if not hist else np.array(x_hist)

In [5]:
v = lambda x: -x[0, 0] * x[1, 0]
h1 = lambda x: -x[0, 0] - x[1, 0]**2 + 1
h2 = lambda x: x[0, 0] + x[1, 0]


def test_aug_lag():
    x0 = np.array([[10], [1]])
    x = augmented_lagrange(x0,
                            cost_function = v,
                            gradient_function = None, 
                            equality_constraints = None, 
                            inequality_constraints = [h1,h2], 
                            tol=1e-4
                            ,tol_const=1e-4)
    return x


In [6]:
test_aug_lag()

[40.]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()